# Api Gateway with Cognito Authentication

This example deploys a microservice with an API Gateway which uses an Amazon Cognito user pool as an authoriser. The POST method on the API Gateway allows authenticated requests only. 

1. The first script is the json formatter, it is used to render json in a readable format.

2. The second script creates a new Cognito user, the user will need to be confirmed.

3. The login in script authenticates the new user, if the authentication is successful Amazon Cognito will return a json web token (jwt)

4. The jwt IdToken is passed to the API Gateway when called the post method. The API Gateway will validate the IdToken against the Cognito user pool that generated it. This is done using an 'Authorizer'.

5. The final script calls an unauthenticated GET method to retrieve the item.

![architecture](../images/architecture_1.png "Architecture")

**json formatter** - Run the following script to create a class which will be used to render json objects in a readable format.

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

 ### Create User
 
 **Set the following variables:**
 
 * clientId (Copy your App client id from Cognito User Pools console)
 * Username
 * Password
 * Email
 
 
 **note:** the user will need to be confirmed either by email or through the cognito console.

In [ ]:
import json, boto3, requests, datetime

# Client ID goes here
clientId = '...'

client = boto3.client('cognito-idp')

signup_response = client.sign_up(
    ClientId=clientId,
    Username='Paul',
    Password='Pa$$word1',
    UserAttributes=[{'Name':'email','Value':'pjcksn@amazon.com'}])

RenderJSON(signup_response)

### Login
Amazon Cognito will return a json web token. The IdToken will be used for authentication against the API Gateway.

**note:** Make sure to confirm the user

In [ ]:
auth_response = client.initiate_auth(
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME' : 'Paul',
        'PASSWORD' : 'Pa$$word1'
    },
    ClientId=clientId
)

print(auth_response['AuthenticationResult'].keys())

### Call API with authentication header and token

The IdToken is passed to the API gateway as a HTTP Authorization header

**Note:** Make sure you set **gwid** to your gateway id using - *gwid = '...'*

In [ ]:
#Set gateway id
gwid = '...'

import boto3, requests, datetime

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')
headers = {'Authorization': auth_response['AuthenticationResult']['IdToken']}

for y in range(2):
    for i in range(2):
        x = datetime.datetime.now()
        accountid = 'a' + str(i)
        vendorid = 'v' + str(i)
        orderdate = str(x)
        coffeetype = 'Short Black'
        coffeesize = 'Small'
        unitprice = str(4.50 * (i + 1))
        quantity = str(i+1)

        response = requests.post(url,json={'order':{
                    'accountid': accountid,
                    'vendorid': vendorid,
                    'orderdate':orderdate,
                    'details':{
                        'coffeetype': coffeetype,
                        'coffeesize': coffeesize,
                        'unitprice': unitprice,
                        'quantity': quantity
                    }
                }
            },headers=headers)
        print(str(accountid) + ':' + str(vendorid))

Get the item from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
response_get = requests.get(url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())